<a href="https://colab.research.google.com/github/BalavSha/Deep-Learning/blob/main/Generative_Adversarial_Networks(GANs).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# <center>**Generative Adversarial Network(GAN)**</center>

## **Text Generation**

Build a sequential model for a **LSTM (Long Short-Term Memory)** neural network:

In [ ]:
# Create a sequential model
regressor = Sequential()

# Add the first LSTM layer with 50 units and 'relu' activation function
# Set 'return_sequences' to True as we will add more LSTM layers
# The input shape is the shape of the training data
regressor.add(LSTM(units=50, activation='relu', return_sequences=True, \
                   input_shape=(X_train.shape[1], 5)))

# Add a dropout layer to prevent overfitting
regressor.add(Dropout(0.2))

# Add the second LSTM layer with 60 units and 'relu' activation function
# Set 'return_sequences' to True as we will add more LSTM layers
regressor.add(LSTM(units=60, activation='relu', return_sequences=True))

# Add another dropout layer
regressor.add(Dropout(0.3))

# Add the third LSTM layer with 80 units and 'relu' activation function
# Set 'return_sequences' to True as we will add more LSTM layers
regressor.add(LSTM(units=80, activation='relu', return_sequences=True))

# Add another dropout layer
regressor.add(Dropout(0.4))

# Add the fourth LSTM layer with 120 units and 'relu' activation function
regressor.add(LSTM(units=120, activation='relu'))

# Add another dropout layer
regressor.add(Dropout(0.5))

# Add a dense layer with 1 unit as the output layer
regressor.add(Dense(units=1))


## **Extending NLP Sequence Models to Generate Text:**

**Steps Involved are:**

*   **Dataset cleaning** encompasses the conversion of the case to lowercase, removing punctuation.

*   **Tokenization** is breaking up a character sequence into specified units called tokens.

*   **Padding** is a way to make input sentences of different sizes the same by padding them.

*   **Padding the sequences** refers to making sure that the sequences have a uniform length.

*   **Stemming** is truncating words down to their stem. For example, the words *rainy* and *raining* both have the stem *rain*.

### ***--> Dataset Cleaning:***

*Create a function to clean text and returns list of words after cleaning. Converting all words in lowercase and encoding for character standardization.*

In [ ]:
# Define a function to clean text
def clean_text(txt):

    # Remove punctuation from the text and convert it to lowercase
    txt = "".join(v for v in txt if v not in string.punctuation).lower()

    # Encode the text as utf-8 and decode it as ascii, ignoring any errors
    txt = txt.encode("utf8").decode("ascii", 'ignore')
    
    # Return the cleaned text
    return txt

# Apply the clean_text function to all headlines in the 'all_headlines' list
corpus = [clean_text(x) for x in all_headlines]

# Print the first 10 elements of the 'corpus' list
corpus[:10]

### ***--> Generating a Sequence and Tokenization:***

In [ ]:
# import tokenizer class from keras
from tensorflow.keras.preprocessing.text import Tokenizer

# Create a Tokenizer object
tokenizer = Tokenizer()

# Define a function to get sequences of tokens from a corpus
def get_seq_of_tokens(corpus):

    # Fit the tokenizer on the corpus
    tokenizer.fit_on_texts(corpus)

    # Get the total number of words in the tokenizer's word index
    all_words = len(tokenizer.word_index) + 1

    # Initialize an empty list to store the input sequences
    input_seq = []

    # Iterate over each line in the corpus
    for line in corpus:

        # Convert the line to a sequence of tokens
        token_list = tokenizer.texts_to_sequences([line])[0]

        # Iterate over each token in the token list
        for i in range(1, len(token_list)):

            # Get the n-gram sequence of tokens
            n_gram_sequence = token_list[:i+1]
            
            # Append the n-gram sequence to the input sequences list
            input_seq.append(n_gram_sequence)

    # Return the input sequences and the total number of words
    return input_seq, all_words

# Get the sequences of tokens and the total number of words from the 'corpus' list
your_sequences, all_words = get_seq_of_tokens(corpus)

# Print the first 10 elements of the 'your_sequences' list
your_sequences[:10]

### ***--> Padding Sequences***

*Create a function to pad each Sequence to make their lengths equal to match length of longest sequence in the text Corpus*

In [ ]:
# Define a function to generate padded sequences
def generate_padded_sequences(input_seq):

    # Get the maximum sequence length
    max_sequence_len = max([len(x) for x in input_seq])

    # Pad the input sequences with zeros, in the beginning of sentences
    input_seq = np.array(pad_sequences(input_seq, maxlen=max_sequence_len, padding='pre'))

    # Split the input sequences into predictors and label
    predictors, label = input_seq[:, :-1], input_seq[:, -1]

    # Convert the label to categorical format
    label = keras.utils.to_categorical(label, num_classes=all_words)

    # Return the predictors, label, and maximum sequence length
    return predictors, label, max_sequence_len

# Generate padded sequences from the 'your_sequences' list
predictors, label, max_sequence_len = generate_padded_sequences(your_sequences)

------

------

#### **Building LSTM Model to generate new news headlines using the historical news information from the dataset**

*Important required libraries*

In [23]:
# Import the pad_sequences function from the Keras library
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Import the Sequential class from the Keras library
from tensorflow.keras.models import Sequential

# Import the Embedding, LSTM, Dense, and Dropout layers from the Keras library
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout

# Import the keras.utils module as ku
import tensorflow.keras.utils as ku

# Import the Tokenizer class from the Keras library
from tensorflow.keras.preprocessing.text import Tokenizer

# Import the pandas library as pd
import pandas as pd
# Import the numpy library as np
import numpy as np

# Import the EarlyStopping callback from the Keras library
from tensorflow.keras.callbacks import EarlyStopping

# Import the string and os modules
import string, os

# Import the warnings module
import warnings
# Ignore all warnings
warnings.filterwarnings("ignore")
warnings.simplefilter(action='ignore', category=FutureWarning)

*Load the Article dataset and display its length:*
--> **Load the Articles.csv dataset in the colab folder first**

In [24]:
# Set the directory path
your_dir = '/content/'

# Initialize an empty list to store the headlines
all_headlines = []

# Iterate over each file in the directory
for filename in os.listdir(your_dir):

    # Check if the filename contains 'Articles'
    if 'Articles' in filename:

        # Read the CSV file into a DataFrame
        article_df = pd.read_csv(your_dir + filename)

        # Extend the headlines list with the values from the 'headline' column
        all_headlines.extend(list(article_df.headline.values))

        # Break out of the loop
        break

# Filter out any headlines with the value 'Unknown'
all_headlines = [h for h in all_headlines if h != "Unknown"]

# Print the length of the 'all_headlines' list
len(all_headlines)

831

*Create a function to return a list of cleaned words. Convert the text to lowercase and encode it with with "utf-8" for character standardization.*

In [25]:
# Define a function to clean text
def clean_text(txt):

    # Remove punctuation from the text and convert it to lowercase
    txt = "".join(v for v in txt if v not in string.punctuation).lower()

    # Encode the text as utf-8 and decode it as ascii, ignoring any errors
    txt = txt.encode("utf8").decode("ascii", 'ignore')

    # Return the cleaned text
    return txt

# Apply the clean_text function to all headlines in the 'all_headlines' list
corpus = [clean_text(x) for x in all_headlines]

# Print headlines of index(60 - 80) of the 'corpus' list
corpus[60:80]

['lets go for a win on opioids',
 'floridas vengeful governor',
 'how to end the politicization of the courts',
 'when dr king came out against vietnam',
 'britains trains dont run on time blame capitalism',
 'questions for no license plates here using art to transcend prison walls',
 'dry spell',
 'are there subjects that should be offlimits to artists or to certain artists in particular',
 'that is great television',
 'thinking in code',
 'how gorsuchs influence could be greater than his vote',
 'new york today how to ease a hangover',
 'trumps gifts to china',
 'at penn station rail mishap spurs large and lasting headache',
 'chemical attack on syrians ignites worlds outrage',
 'adventure is still on babbos menu',
 'swimming in the fast lane',
 'a national civics exam',
 'obama adviser is back in the political cross hairs',
 'the hippies have won']

*Create a function to convert a given corpus of text into a list of input sequences of tokens. It does this by converting each line in the corpus into a sequence of tokens.*

In [26]:
# Create a Tokenizer object
tokenizer = Tokenizer()

# Define a function to get sequences of tokens from a corpus
def get_seq_of_tokens(corpus):

    # Fit the tokenizer on the corpus
    tokenizer.fit_on_texts(corpus)

    # Get the total number of words in the tokenizer's word index
    all_words = len(tokenizer.word_index) + 1

    # Initialize an empty list to store the input sequences
    input_seq = []

    # Iterate over each line in the corpus
    for line in corpus:

        # Convert each line to a sequence of tokens
        token_list = tokenizer.texts_to_sequences([line])[0]

        # Iterate over each token in the token list
        for i in range(1, len(token_list)):

            # Get the n-gram sequence of tokens
            n_gram_sequence = token_list[:i+1]

            # Append the n-gram sequence to the input sequences list
            input_seq.append(n_gram_sequence)

    # Return the input sequences and the total number of words
    return input_seq, all_words

# Get the sequences of tokens and the total number of words from the 'corpus' list
your_sequences, all_words = get_seq_of_tokens(corpus)

# Print the first 20 elements of the 'your_sequences' list
your_sequences[:20]

[[169, 17],
 [169, 17, 665],
 [169, 17, 665, 367],
 [169, 17, 665, 367, 4],
 [169, 17, 665, 367, 4, 2],
 [169, 17, 665, 367, 4, 2, 666],
 [169, 17, 665, 367, 4, 2, 666, 170],
 [169, 17, 665, 367, 4, 2, 666, 170, 5],
 [169, 17, 665, 367, 4, 2, 666, 170, 5, 667],
 [6, 80],
 [6, 80, 1],
 [6, 80, 1, 668],
 [6, 80, 1, 668, 10],
 [6, 80, 1, 668, 10, 669],
 [670, 671],
 [670, 671, 129],
 [670, 671, 129, 672],
 [673, 674],
 [673, 674, 368],
 [673, 674, 368, 675]]

*Create a function to pad each Sequence to make their lengths equal to match length of longest sequence in the text Corpus:*

In [27]:
# Define a function to generate padded sequences
def generate_padded_sequences(input_seq):

    # Get the maximum sequence length
    max_sequence_len = max([len(x) for x in input_seq])

    # Pad the input sequences with zeros in the beginning
    input_seq = np.array(pad_sequences(input_seq, maxlen=max_sequence_len, padding='pre'))

    # Split the input sequences into predictors and label
    predictors, label = input_seq[:, :-1], input_seq[:, -1]

    # Convert the label to categorical format
    label = ku.to_categorical(label, num_classes=all_words)

    # Return the predictors, label, and maximum sequence length
    return predictors, label, max_sequence_len

# Generate padded sequences from the 'inp_seq' list
predictors, label, max_sequence_len = generate_padded_sequences(your_sequences)

*Prepare the Model architecture for compiling and training:*

In [28]:
# define a function to create the model
def create_model(max_sequence_len, all_words):

  # set the input length
  input_len = max_sequence_len - 1

  # create a Sequential model
  model = Sequential()

  # add embedding layer to create word vectors for incoming words
  model.add(Embedding(all_words, 10, input_length=input_len))

  # add an LSTM layer with 100 units
  model.add(LSTM(100))

  # add a dropout layer to prevent overfitting
  model.add(Dropout(0.1))

  # add a output Dense layer with "all_words" units and "softmax" activation function
  model.add(Dense(all_words, activation="softmax"))

  # compile the model with "categorical_crossentropy" and "Adam" optimizer
  model.compile(
      optimizer = "adam",
      loss = "categorical_crossentropy"
              )
  
  # return the model
  return model

# create the model using the "max_sequence_len" and "all_words" variables
model = create_model(max_sequence_len, all_words)

In [29]:
# display the summary of model architecture
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 18, 10)            24220     
                                                                 
 lstm_1 (LSTM)               (None, 100)               44400     
                                                                 
 dropout_1 (Dropout)         (None, 100)               0         
                                                                 
 dense_1 (Dense)             (None, 2422)              244622    
                                                                 
Total params: 313,242
Trainable params: 313,242
Non-trainable params: 0
_________________________________________________________________


*Fit the model with data*

In [30]:
model.fit(
    predictors,
    label,
    epochs = 200,
    verbose = 5
)

Epoch 1/200
Epoch 2/200
Epoch 3/200
Epoch 4/200
Epoch 5/200
Epoch 6/200
Epoch 7/200
Epoch 8/200
Epoch 9/200
Epoch 10/200
Epoch 11/200
Epoch 12/200
Epoch 13/200
Epoch 14/200
Epoch 15/200
Epoch 16/200
Epoch 17/200
Epoch 18/200
Epoch 19/200
Epoch 20/200
Epoch 21/200
Epoch 22/200
Epoch 23/200
Epoch 24/200
Epoch 25/200
Epoch 26/200
Epoch 27/200
Epoch 28/200
Epoch 29/200
Epoch 30/200
Epoch 31/200
Epoch 32/200
Epoch 33/200
Epoch 34/200
Epoch 35/200
Epoch 36/200
Epoch 37/200
Epoch 38/200
Epoch 39/200
Epoch 40/200
Epoch 41/200
Epoch 42/200
Epoch 43/200
Epoch 44/200
Epoch 45/200
Epoch 46/200
Epoch 47/200
Epoch 48/200
Epoch 49/200
Epoch 50/200
Epoch 51/200
Epoch 52/200
Epoch 53/200
Epoch 54/200
Epoch 55/200
Epoch 56/200
Epoch 57/200
Epoch 58/200
Epoch 59/200
Epoch 60/200
Epoch 61/200
Epoch 62/200
Epoch 63/200
Epoch 64/200
Epoch 65/200
Epoch 66/200
Epoch 67/200
Epoch 68/200
Epoch 69/200
Epoch 70/200
Epoch 71/200
Epoch 72/200
Epoch 73/200
Epoch 74/200
Epoch 75/200
Epoch 76/200
Epoch 77/200
Epoch 78

**Create a function that will generate a headline given a starting seed text, the number of words to generate, the model, and the maximum sequence length.** <br>

*The function will include a for loop to iterate over the number of words to generate.* <br>

*In each iteration, the tokenizer will tokenize the text, and then pad the sequence before predicting the next word in the sequence.* <br>

*Next, the iteration will convert the token back into a word and add it to the sentence.* <br>

*Once the for loop completes, the generated headline will be returned:*

In [34]:
# Define a function to generate text
def generate_text(seed_text, next_words, model, max_sequence_len):

    # Iterate for the number of next words to generate
    for _ in range(next_words):

        # Convert the seed text to a sequence of tokens
        token_list = tokenizer.texts_to_sequences([seed_text])[0]

        # Pad the token list with zeros
        token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')

        # Make a prediction using the model
        predicted = model.predict(token_list, verbose=0)

        # Initialize an empty string to store the output word
        output_word = ""

        # Iterate over each word and index in the tokenizer's word index
        for word, index in tokenizer.word_index.items():

            # Check if the index matches the predicted value
            if index == np.argmax(predicted):

                # Set the output word to the current word
                output_word = word

                # Break out of the loop
                break

        # Append the output word to the seed text
        seed_text += " " + output_word

    # Return the generated text with title casing
    return seed_text.title()

*Use the function above to output text generation as below:*

In [46]:
# Generate text using the 'generate_text' function and print the result
print(generate_text("Balav is a Machine Learning", 5, model, max_sequence_len))
print()
print(generate_text("europe looks to", 8, model, max_sequence_len))
print()
print(generate_text("best way", 10, model, max_sequence_len))
print()
print(generate_text("homeless in", 10, model, max_sequence_len))
print()
print(generate_text("unexpected results", 10, model, max_sequence_len))
print()
print(generate_text("Are there subjects that should be offlimits to artists or to certain artists in", 1, model, max_sequence_len))

Balav Is A Machine Learning Of The Border On Official

Europe Looks To Feel Really American Try Some Velveeta To Use

Best Way To Move Forward Sets May You 2017 Warm 2017 Men

Homeless In The Center Of A Classic Nasty Woman On An Elegy

Unexpected Results Isnt That Could Expand Access To Fossil Fuels Thinks Nuisance

Are There Subjects That Should Be Offlimits To Artists Or To Certain Artists In Particular


------

------

# <center>**Generative Adversarial Networks(GANs)**</center>

**--> GANs are networks that generates new, synthetic data by learning patterns and underlying representations from a training dataset.**

--> For instance, generating a new Images based on a given Images dataset.